In [2]:
import numpy as np
import pandas as pd
from scipy.stats import f, norm, pearsonr, ttest_rel

!pip install --upgrade gensim
import re
from itertools import permutations

import gensim.downloader as api
import scipy.stats as stats
from gensim.models import KeyedVectors
from scipy.spatial.distance import cosine, euclidean, pdist

# Load data
study_name = "idea generation"
specification_name = "default persona"
human_file = f"{study_name} human data labels anonymized.csv"
twin_file = f"{study_name} twins data labels anonymized.csv"
df_human = pd.read_csv(human_file, header=0, skiprows=[1, 2])
df_twin = pd.read_csv(twin_file, header=0, skiprows=[1])

# creativity ratings:
human_file = f"{study_name} human data creativity ratings.csv"
twin_file = f"{study_name} twins data creativity ratings.csv"
creativity_human = pd.read_csv(human_file, header=0)
creativity_twin = pd.read_csv(twin_file, header=0)

# pull out just the ID + creativity columns
creat_col = creativity_human[
    [
        "TWIN_ID",
        "creativity_rating_byhuman",
        "creativity_rating_byhuman_partial",
        "creativity_rating_byhuman_full",
        "creativity_rating_bytwins",
    ]
]
# merge into df_human (left‐join so you keep all rows in df_human)
df_human = df_human.merge(creat_col, on="TWIN_ID", how="left")
creat_col = creativity_twin[
    [
        "TWIN_ID",
        "creativity_rating_byhuman",
        "creativity_rating_byhuman_partial",
        "creativity_rating_byhuman_full",
        "creativity_rating_bytwins",
    ]
]
# merge into df_human (left‐join so you keep all rows in df_human)
df_twin = df_twin.merge(creat_col, on="TWIN_ID", how="left")

# previous human data:
wave1_human = pd.read_csv("wave 1 scores.csv", header=0)
wave2_human = pd.read_csv("wave 2 scores.csv", header=0)
wave3_human = pd.read_csv("wave 3 scores.csv", header=0)
# pull out relevant columns
creat_col = wave2_human[["TWIN_ID", "score_forwardflow"]]
# merge into df_human (left‐join so you keep all rows in df_human)
df_human = df_human.merge(creat_col, on="TWIN_ID", how="left")
# add also to twins, for symmetry:
df_twin = df_twin.merge(creat_col, on="TWIN_ID", how="left")
# Load the Word2Vec model (Google News model; this will download the model if not already available)
print("Loading word2vec model... (this may take a while)")
model = api.load("word2vec-google-news-300")
print("Model loaded.")


#########################
# create new relevant columns: DAT
text_cols = [f"Q74_{i}" for i in range(1, 11)]


def compute_dat_perf_from_cols(row):
    # 1) grab the 10 entries, lowercase and drop any NaN/empty
    tokens = [
        str(row[col]).lower().strip()
        for col in text_cols
        if pd.notna(row[col]) and str(row[col]).strip() != ""
    ]
    # 2) look up embeddings (skip tokens not in vocab)
    vecs = [model[w] for w in tokens if w in model.key_to_index]
    # 3) must have at least 7 embeddings
    if len(vecs) < 7:
        return np.nan
    # 4) stack the first 7 into a matrix
    mat = np.vstack(vecs[:7])
    # 5) compute all pairwise Euclidean distances and return the mean
    #    dists = pdist(mat, metric='euclidean')
    dists = pdist(mat, metric="cosine")
    return dists.mean()


# apply it directly over the 10 columns
df_human["DAT_perf"] = df_human[text_cols].apply(compute_dat_perf_from_cols, axis=1)
df_twin["DAT_perf"] = df_twin[text_cols].apply(compute_dat_perf_from_cols, axis=1)
# quick check
print(df_human[["DAT_perf"]].head())
print(df_twin[["DAT_perf"]].head())
###############################


#########################
# create new relevant columns: SSPT
def compute_circuitousness_euclidean(words, model):
    """Euclidean‐based circuitousness over 5 word embeddings."""
    if len(words) != len(set(words)):
        return np.nan
    embs = []
    for w in words:
        toks = [t for t in str(w).lower().split() if t]
        vecs = [model[t] for t in toks if t in model.key_to_index]
        if not vecs:
            return np.nan
        embs.append(np.mean(vecs, axis=0))
    if len(embs) != 5:
        return np.nan
    # original path
    orig = sum(euclidean(embs[i], embs[i + 1]) for i in range(4))
    best = np.inf
    for perm in permutations(embs[1:4]):
        path = [embs[0]] + list(perm) + [embs[4]]
        length = sum(euclidean(path[i], path[i + 1]) for i in range(4))
        best = min(best, length)
    return orig / best if best > 0 else np.nan


def compute_circuitousness_cosine(words, model):
    """Cosine‐based circuitousness over 5 word embeddings."""
    if len(words) != len(set(words)):
        return np.nan
    embs = []
    for w in words:
        toks = [t for t in str(w).lower().split() if t]
        vecs = [model[t] for t in toks if t in model.key_to_index]
        if not vecs:
            return np.nan
        embs.append(np.mean(vecs, axis=0))
    if len(embs) != 5:
        return np.nan
    orig = sum(cosine(embs[i], embs[i + 1]) for i in range(4))
    best = np.inf
    for perm in permutations(embs[1:4]):
        path = [embs[0]] + list(perm) + [embs[4]]
        length = sum(cosine(path[i], path[i + 1]) for i in range(4))
        best = min(best, length)
    return orig / best if best > 0 else np.nan


def add_sspt_all(df, model):
    prefixes = [1, 2, 4, 5, 7]
    eu_cols, co_cols = [], []

    # compute raw circuitousness for each metric & task
    for p in prefixes:
        cols = [f"{p}_Q30_{i}" for i in range(1, 6)]
        df[cols] = df[cols].applymap(lambda x: str(x).lower())

        e_col = f"circuitousness_task{p}_euclid"
        c_col = f"circuitousness_task{p}_cosine"
        eu_cols.append(e_col)
        co_cols.append(c_col)

        df[e_col] = df[cols].apply(
            lambda r: compute_circuitousness_euclidean(r.tolist(), model), axis=1
        )
        df[c_col] = df[cols].apply(
            lambda r: compute_circuitousness_cosine(r.tolist(), model), axis=1
        )

    # raw SSPT
    df["SSPT"] = df[eu_cols].mean(axis=1)
    df["n_non_na_tasks_SSPT"] = df[eu_cols].notna().sum(axis=1)
    df["SSPT_cosine"] = df[co_cols].mean(axis=1)
    df["n_non_na_tasks_SSPT_cosine"] = df[co_cols].notna().sum(axis=1)

    # standardize per task
    stats_eu = df[eu_cols].agg(["mean", "std"])
    stats_co = df[co_cols].agg(["mean", "std"])

    z_eu, z_co = [], []
    for col in eu_cols:
        zc = f"{col}_z"
        df[zc] = (df[col] - stats_eu.loc["mean", col]) / stats_eu.loc["std", col]
        z_eu.append(zc)
    for col in co_cols:
        zc = f"{col}_z"
        df[zc] = (df[col] - stats_co.loc["mean", col]) / stats_co.loc["std", col]
        z_co.append(zc)

    # standardized SSPT
    df["SSPT_standardized"] = df[z_eu].mean(axis=1)
    df["n_non_na_tasks_SSPT_standardized"] = df[z_eu].notna().sum(axis=1)
    df["SSPT_standardized_cosine"] = df[z_co].mean(axis=1)
    df["n_non_na_tasks_SSPT_standardized_cosine"] = df[z_co].notna().sum(axis=1)

    return df


# Apply to both datasets
df_human = add_sspt_all(df_human, model)
df_twin = add_sspt_all(df_twin, model)

# Quick peek
print(df_human[["SSPT", "SSPT_standardized", "SSPT_cosine", "SSPT_standardized_cosine"]].head())

print(df_twin[["SSPT", "SSPT_standardized", "SSPT_cosine", "SSPT_standardized_cosine"]].head())


#########################################


# Remove any leading apostrophes and coerce to float
df_human["DAT_perf"] = pd.to_numeric(
    df_human["DAT_perf"].astype(str).str.lstrip("'"), errors="coerce"
)

df_twin["DAT_perf"] = pd.to_numeric(
    df_twin["DAT_perf"].astype(str).str.lstrip("'"), errors="coerce"
)

# Do the same for any other columns you expect to be numeric
for col in ["SSPT", "creativity_rating_byhuman"] + [
    c for c in df_human if c.startswith("circuitousness_task")
]:
    df_human[col] = pd.to_numeric(df_human[col], errors="coerce")
    df_twin[col] = pd.to_numeric(df_twin[col], errors="coerce")

# Confirm
print(df_human[["DAT_perf", "SSPT", "creativity_rating_byhuman"]].dtypes)

df_twin.to_csv("idea_generation_twins_processed.csv", index=False)
df_human.to_csv("idea_generation_human_processed.csv", index=False)

df_human = df_human.set_index("TWIN_ID")
df_twin = df_twin.set_index("TWIN_ID")


##################now proceed with standard meta-analysis steps
# define relevant columns:
# condition variable names:
condition_vars = [""]
# Check if we have a real condition var
if condition_vars and condition_vars[0].strip():
    cond = condition_vars[0]
    cond_h = f"{cond}_human"
    cond_t = f"{cond}_twin"
    cond_exists = True
else:
    cond_exists = False

# raw responses:
# raw_vars = []
# raw_vars_min = [1]*20
# raw_vars_max = [5]*20
# raw responses: domain=social?
# raw_vars_social=[1]*20
# raw_vars_social_map = dict(zip(raw_vars, raw_vars_social))
# raw responses: domain=cognitive?
# raw_vars_cognitive=[0]*20
# raw_vars_cognitive_map = dict(zip(raw_vars, raw_vars_cognitive))
# raw responses: replicating know human bias?
# raw_vars_known=[0]*20
# raw_vars_known_map = dict(zip(raw_vars, raw_vars_known))
# raw responses: preference measure?
# raw_vars_pref=[1]*20
# raw_vars_pref_map = dict(zip(raw_vars, raw_vars_pref))
# raw responses: stimuli dependent?
# raw_vars_stim=[1]*20
# raw_vars_stim_map = dict(zip(raw_vars, raw_vars_stim))

# DVs:
DV_vars = ["DAT_perf", "SSPT", "creativity_rating_byhuman"]
DV_vars_min = ["nan", "nan", 1]
DV_vars_max = ["nan", "nan", 5]
# DVs: domain=social?
DV_vars_social = [0] * 3
DV_vars_social_map = dict(zip(DV_vars, DV_vars_social))
# DVs: domain=cognitive?
DV_vars_cognitive = [1] * 3
DV_vars_cognitive_map = dict(zip(DV_vars, DV_vars_cognitive))
# DVs: replicating know human bias?
DV_vars_known = [0] * 3
DV_vars_known_map = dict(zip(DV_vars, DV_vars_known))
# DVs: preference measure?
DV_vars_pref = [0] * 3
DV_vars_pref_map = dict(zip(DV_vars, DV_vars_pref))
# DVs: stimuli dependent?
DV_vars_stim = [0] * 3
DV_vars_stim_map = dict(zip(DV_vars, DV_vars_stim))
# DVs: knowledge question?
DV_vars_know = [0] * 3
DV_vars_know_map = dict(zip(DV_vars, DV_vars_know))
# DVs: political question?
DV_vars_politics = [0] * 3
DV_vars_politics_map = dict(zip(DV_vars, DV_vars_politics))

# merging key
merge_key = ["TWIN_ID"]

# Merge on TWIN_ID
df = pd.merge(df_human, df_twin, on=merge_key, suffixes=("_human", "_twin"))

# Fix dtypes
# for var in raw_vars + DV_vars:
for var in DV_vars:
    df[f"{var}_human"] = pd.to_numeric(df[f"{var}_human"], errors="coerce")
    df[f"{var}_twin"] = pd.to_numeric(df[f"{var}_twin"], errors="coerce")

# build min/max maps from both raw_vars and DV_vars
min_map = {v: mn for v, mn in zip(DV_vars, DV_vars_min)}
# min_map = {v: mn for v, mn in zip(raw_vars,      raw_vars_min)}
# min_map.update({v: mn for v, mn in zip(DV_vars,   DV_vars_min)})

max_map = {v: mx for v, mx in zip(DV_vars, DV_vars_max)}
# max_map = {v: mx for v, mx in zip(raw_vars,      raw_vars_max)}
# max_map.update({v: mx for v, mx in zip(DV_vars,   DV_vars_max)})

# now add _min and _max columns for every variable in the union
for var in min_map:
    df[f"{var}_min"] = min_map[var]
    df[f"{var}_max"] = max_map[var]

# Compute results
results = []
# for var in raw_vars:
#     col_h = f"{var}_human"
#     col_t = f"{var}_twin"
#     min_col = f"{var}_min"
#     max_col = f"{var}_max"
#     if cond_exists:
#         cols = [col_h, col_t, cond_h, cond_t,min_col,max_col]
#     else:
#         cols = [col_h, col_t,min_col,max_col]
#     pair = (
#     df[cols]
#       .dropna(subset=[col_h, col_t])
#     )
#     min_val = pair[min_col].iloc[0]
#     max_val = pair[max_col].iloc[0]
#     n    = len(pair)
#     if n >= 4:
#         r, _    = pearsonr(pair[col_h], pair[col_t])
#         z_f     = np.arctanh(r)
#         se      = 1 / np.sqrt(n - 3)
#         z_crit  = norm.ppf(0.975)
#         lo_z, hi_z = z_f - z_crit*se, z_f + z_crit*se
#         lo_r, hi_r = np.tanh(lo_z), np.tanh(hi_z)
#         z_score    = z_f / se
#         # Accuracy = mean absolute diff / range
#         if pd.isna(min_val) or pd.isna(max_val) or max_val == min_val:
#             accuracy = np.nan
#         else:
#             # compute mean absolute difference
#             abs_diff      = np.abs(pair[col_h] - pair[col_t])
#             mean_abs_diff = abs_diff.mean()
#             accuracy      = 1 - mean_abs_diff / (max_val - min_val)

#         mean_h = pair[col_h].mean()
#         mean_t = pair[col_t].mean()

#         # Paired t‐test
#         t_stat, p_val = ttest_rel(pair[col_h], pair[col_t])

#         std_h = pair[col_h].std(ddof=1)
#         std_t = pair[col_t].std(ddof=1)

#          # F‐test for equal variances
#         df1 = df2 = n - 1
#         f_stat = (std_h**2 / std_t**2) if std_t>0 else np.nan

#         # two‐tailed p‐value:
#         if not np.isnan(f_stat):
#             p_f = 2 * min(f.cdf(f_stat, df1, df2),
#                           1 - f.cdf(f_stat, df1, df2))
#         else:
#             p_f = np.nan

#         # Effect sizes (Cohen's d) across conditions
#         #    For humans:
#         if cond_exists and len(pair)>3:
#             levels_h = pair[cond_h].unique()
#             if len(levels_h) == 2:
#                 g1 = pair.loc[pair[cond_h]==levels_h[0], col_h]
#                 g2 = pair.loc[pair[cond_h]==levels_h[1], col_h]
#                 n1, n2 = len(g1), len(g2)
#                 # pooled sd
#                 s_pool = np.sqrt(((n1-1)*g1.var(ddof=1)+(n2-1)*g2.var(ddof=1)) / (n1+n2-2))
#                 d_human = (g1.mean() - g2.mean()) / s_pool if s_pool>0 else np.nan
#             else:
#                 d_human = np.nan
#         else:
#             d_human = np.nan

#         #    For twins:
#         if cond_exists and len(pair)>3:
#             levels_t = pair[cond_t].unique()
#             if cond_exists and len(levels_t) == 2:
#                 g1 = pair.loc[pair[cond_t]==levels_t[0], col_t]
#                 g2 = pair.loc[pair[cond_t]==levels_t[1], col_t]
#                 n1, n2 = len(g1), len(g2)
#                 s_pool = np.sqrt(((n1-1)*g1.var(ddof=1)+(n2-1)*g2.var(ddof=1)) / (n1+n2-2))
#                 d_twin = (g1.mean() - g2.mean()) / s_pool if s_pool>0 else np.nan
#             else:
#                 d_twin = np.nan
#         else:
#             d_twin = np.nan
#     else:
#         r = lo_r = hi_r = z_score = accuracy = mean_h = mean_t = t_stat = p_val = std_h = std_t = f_stat = p_f = np.nan
#         d_human = d_twin = np.nan


#     results.append({
#         'study name': study_name,
#         'variable name': var,
#         'variable type (raw response/DV)':     'raw',
#         'correlation between the responses from humans vs. their twins':        r,
#         'CI_lower': lo_r,
#         'CI_upper': hi_r,
#         'z-score for correlation between humans vs. their twins':  z_score,
#         'accuracy between humans vs. their twins': accuracy,
#         'mean_human': mean_h,
#         'mean_twin': mean_t,
#         'paired t-test t-stat': t_stat,
#         'paired t-test p-value': p_val,
#         'std_human': std_h,
#         'std_twin': std_t,
#         'variance test F-stat': f_stat,
#         'variance test p-value': p_f,
#         'effect size based on human': d_human,
#         'effect size based on twin': d_twin,
#         'domain=social?':raw_vars_social_map.get(var, np.nan),
#         'domain=cognitive?':raw_vars_cognitive_map.get(var, np.nan),
#         'replicating know human bias?':raw_vars_known_map.get(var, np.nan),
#         'preference measure?':raw_vars_pref_map.get(var, np.nan),
#         'stimuli dependent?':raw_vars_stim_map.get(var, np.nan),
#         'sample size':        n
#     })

for var in DV_vars:
    col_h = f"{var}_human"
    col_t = f"{var}_twin"
    min_col = f"{var}_min"
    max_col = f"{var}_max"
    if cond_exists:
        cols = [col_h, col_t, cond_h, cond_t, min_col, max_col]
    else:
        cols = [col_h, col_t, min_col, max_col]
    pair = df[cols].dropna(subset=[col_h, col_t])
    min_val = pair[min_col].iloc[0]
    max_val = pair[max_col].iloc[0]
    n = len(pair)
    if n >= 4:
        r, _ = pearsonr(pair[col_h], pair[col_t])
        z_f = np.arctanh(r)
        se = 1 / np.sqrt(n - 3)
        z_crit = norm.ppf(0.975)
        lo_z, hi_z = z_f - z_crit * se, z_f + z_crit * se
        lo_r, hi_r = np.tanh(lo_z), np.tanh(hi_z)
        z_score = z_f / se
        # Accuracy = mean absolute diff / range
        if pd.isna(min_val) or pd.isna(max_val) or max_val == min_val:
            accuracy = np.nan
        else:
            # compute mean absolute difference
            abs_diff = np.abs(pair[col_h] - pair[col_t])
            mean_abs_diff = abs_diff.mean()
            accuracy = 1 - mean_abs_diff / (max_val - min_val)

        mean_h = pair[col_h].mean()
        mean_t = pair[col_t].mean()

        # Paired t‐test
        t_stat, p_val = ttest_rel(pair[col_h], pair[col_t])

        std_h = pair[col_h].std(ddof=1)
        std_t = pair[col_t].std(ddof=1)

        # F‐test for equal variances
        df1 = df2 = n - 1
        f_stat = (std_h**2 / std_t**2) if std_t > 0 else np.nan
        # two‐tailed p‐value:
        if not np.isnan(f_stat):
            p_f = 2 * min(f.cdf(f_stat, df1, df2), 1 - f.cdf(f_stat, df1, df2))
        else:
            p_f = np.nan

        # Effect sizes (Cohen's d) across conditions
        #    For humans:
        if cond_exists and len(pair) > 3:
            levels_h = pair[cond_h].unique()
            if len(levels_h) == 2:
                g1 = pair.loc[pair[cond_h] == levels_h[0], col_h]
                g2 = pair.loc[pair[cond_h] == levels_h[1], col_h]
                n1, n2 = len(g1), len(g2)
                # pooled sd
                s_pool = np.sqrt(
                    ((n1 - 1) * g1.var(ddof=1) + (n2 - 1) * g2.var(ddof=1)) / (n1 + n2 - 2)
                )
                d_human = (g1.mean() - g2.mean()) / s_pool if s_pool > 0 else np.nan
            else:
                d_human = np.nan
        else:
            d_human = np.nan

        #    For twins:
        if cond_exists and len(pair) > 3:
            levels_t = pair[cond_t].unique()
            if cond_exists and len(levels_t) == 2:
                g1 = pair.loc[pair[cond_t] == levels_t[0], col_t]
                g2 = pair.loc[pair[cond_t] == levels_t[1], col_t]
                n1, n2 = len(g1), len(g2)
                s_pool = np.sqrt(
                    ((n1 - 1) * g1.var(ddof=1) + (n2 - 1) * g2.var(ddof=1)) / (n1 + n2 - 2)
                )
                d_twin = (g1.mean() - g2.mean()) / s_pool if s_pool > 0 else np.nan
            else:
                d_twin = np.nan
        else:
            d_twin = np.nan
    else:
        r = lo_r = hi_r = z_score = accuracy = mean_h = mean_t = t_stat = p_val = std_h = std_t = (
            f_stat
        ) = p_f = np.nan
        d_human = d_twin = np.nan

    results.append(
        {
            "study name": study_name,
            "persona specification": specification_name,
            "variable name": var,
            #        'variable type (raw response/DV)':     'DV',
            "correlation between the responses from humans vs. their twins": r,
            "CI_lower": lo_r,
            "CI_upper": hi_r,
            "z-score for correlation between humans vs. their twins": z_score,
            "accuracy between humans vs. their twins": accuracy,
            "mean_human": mean_h,
            "mean_twin": mean_t,
            "paired t-test t-stat": t_stat,
            "paired t-test p-value": p_val,
            "std_human": std_h,
            "std_twin": std_t,
            "variance test F-stat": f_stat,
            "variance test p-value": p_f,
            "effect size based on human": d_human,
            "effect size based on twin": d_twin,
            "domain=social?": DV_vars_social_map.get(var, np.nan),
            "domain=cognitive?": DV_vars_cognitive_map.get(var, np.nan),
            "replicating know human bias?": DV_vars_known_map.get(var, np.nan),
            "preference measure?": DV_vars_pref_map.get(var, np.nan),
            "stimuli dependent?": DV_vars_stim_map.get(var, np.nan),
            "knowledge question?": DV_vars_know_map.get(var, np.nan),
            "political question?": DV_vars_politics_map.get(var, np.nan),
            "sample size": n,
        }
    )

# results DataFrame
corr_df = pd.DataFrame(results)
print(corr_df)

# save output as csv - unit of observation is comparison between humans and twins:
out_file = f"{study_name} {specification_name} meta analysis.csv"
corr_df.to_csv(out_file, index=False)


#####participant-level data:
def make_long(df, respondent_type):
    # pick off TWIN_ID + the DVs, then melt
    long = df[["TWIN_ID"] + DV_vars].melt(
        id_vars="TWIN_ID", value_vars=DV_vars, var_name="variable_name", value_name="value"
    )

    long["respondent_type"] = respondent_type
    long["study_name"] = study_name
    long["specification_name"] = specification_name
    return long


#########################unique to this study:
# turn the index back into a column so melt can see it
df_human.reset_index(inplace=True)
df_twin.reset_index(inplace=True)
###################

# build the two halves
long_h = make_long(df_human, "human")
long_t = make_long(df_twin, "twin")

# stack them
df_long = pd.concat([long_h, long_t], ignore_index=True)

print(df_long.head())
# save output as csv - unit of observation is TWIN_ID:
out_file = f"{study_name} {specification_name} meta analysis individual level.csv"
df_long.to_csv(out_file, index=False)

print("done")

Loading word2vec model... (this may take a while)
Model loaded.
   DAT_perf
0  0.840487
1  0.843097
2  0.872506
3  0.799434
4  0.908836
   DAT_perf
0  0.913294
1  0.880396
2  0.877118
3  0.899895
4  0.933089


C:\Users\ot2107\AppData\Local\Temp\ipykernel_30244\4047731639.py:149: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols] = df[cols].applymap(lambda x: str(x).lower())
C:\Users\ot2107\AppData\Local\Temp\ipykernel_30244\4047731639.py:149: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols] = df[cols].applymap(lambda x: str(x).lower())
C:\Users\ot2107\AppData\Local\Temp\ipykernel_30244\4047731639.py:149: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols] = df[cols].applymap(lambda x: str(x).lower())
C:\Users\ot2107\AppData\Local\Temp\ipykernel_30244\4047731639.py:149: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols] = df[cols].applymap(lambda x: str(x).lower())
C:\Users\ot2107\AppData\Local\Temp\ipykernel_30244\4047731639.py:149: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols] =

       SSPT  SSPT_standardized  SSPT_cosine  SSPT_standardized_cosine
0  1.013209           0.009591     1.026783                  0.073893
1  1.035666           0.960328     1.077662                  1.159764
2  1.008889          -0.075669     1.019849                 -0.020189
3  1.013648           0.101516     1.016054                 -0.129368
4  1.011115          -0.003051     1.023007                  0.032124
       SSPT  SSPT_standardized  SSPT_cosine  SSPT_standardized_cosine
0  1.015385           0.092685     1.026062                  0.101128
1  1.017950          -0.026612     1.019446                 -0.153549
2  1.010380           0.013762     1.018857                 -0.030102
3  1.023293           0.313542     1.044213                  0.521580
4  1.000822          -0.510037     1.000000                 -0.507717
DAT_perf                     float64
SSPT                         float64
creativity_rating_byhuman    float64
dtype: object
        study name persona specific

In [ ]:
# test: use cosine distance instead of Euclidean distance in SSPT:

In [69]:
# replicate pre-registered analysis:
from scipy.stats import pearsonr

# 1) define the exact order you want
vars_list_human = [
    "DAT_perf",
    "score_forwardflow",
    "SSPT",
    "SSPT_cosine",
    "SSPT_standardized",
    "SSPT_standardized_cosine",
    "creativity_rating_byhuman",
    "creativity_rating_byhuman_partial",
    "creativity_rating_byhuman_full",
]
human_cols = [f"{v}_human" for v in vars_list_human]
vars_list_twin = [
    "DAT_perf",
    "SSPT",
    "SSPT_cosine",
    "SSPT_standardized",
    "SSPT_standardized_cosine",
    "creativity_rating_bytwins",
    "creativity_rating_byhuman",
    "creativity_rating_byhuman_partial",
    "creativity_rating_byhuman_full",
]
twin_cols = [f"{v}_twin" for v in vars_list_twin]
all_cols = human_cols + twin_cols

# --- 2) Means summary, in the same order ---
mean_cols = [f"{v}_human_mean" for v in vars_list_human] + [
    f"{v}_twin_mean" for v in vars_list_twin
]
means = {
    mc: df[col[:-5]].mean()  # strip _mean to get var_human/var_twin
    for mc, col in zip(mean_cols, mean_cols)
}
means_df = pd.DataFrame([means], columns=mean_cols)
means_df.to_csv(f"{study_name}_means_summary.csv", index=False)
print("→ means summary written with columns:", mean_cols)

# standard deviations
std_cols = [f"{v}_human_std" for v in vars_list_human] + [f"{v}_twin_std" for v in vars_list_twin]
stds = {}
# humans
for v, col in zip(vars_list_human, std_cols[: len(vars_list_human)]):
    stds[col] = df[f"{v}_human"].std(ddof=1)
# twins
for v, col in zip(vars_list_twin, std_cols[len(vars_list_human) :]):
    stds[col] = df[f"{v}_twin"].std(ddof=1)
stds_df = pd.DataFrame([stds], columns=std_cols)
stds_df.to_csv(f"{study_name}_std_summary.csv", index=False)
print("→ std‐summary written with columns:", std_cols)

# --- 3) Correlation matrix, rows & columns in the same order ---
corr_mat = df[all_cols].corr().loc[all_cols, all_cols]
corr_mat.to_csv(f"{study_name}_correlation_matrix.csv")
print("→ correlation matrix written with index/cols:", all_cols)

# --- 4) P-value matrix, rows & columns in the same order ---
pval_mat = pd.DataFrame(np.nan, index=all_cols, columns=all_cols)
for i in all_cols:
    for j in all_cols:
        x, y = df[i], df[j]
        mask = x.notna() & y.notna()
        if mask.sum() > 1:
            _, p = pearsonr(x[mask], y[mask])
            pval_mat.at[i, j] = p
pval_mat = pval_mat.loc[all_cols, all_cols]
pval_mat.to_csv(f"{study_name}_pvalues_matrix.csv")
print("→ p-value matrix written with index/cols:", all_cols)

→ means summary written with columns: ['DAT_perf_human_mean', 'score_forwardflow_human_mean', 'SSPT_human_mean', 'SSPT_cosine_human_mean', 'SSPT_standardized_human_mean', 'SSPT_standardized_cosine_human_mean', 'creativity_rating_byhuman_human_mean', 'creativity_rating_byhuman_partial_human_mean', 'creativity_rating_byhuman_full_human_mean', 'DAT_perf_twin_mean', 'SSPT_twin_mean', 'SSPT_cosine_twin_mean', 'SSPT_standardized_twin_mean', 'SSPT_standardized_cosine_twin_mean', 'creativity_rating_bytwins_twin_mean', 'creativity_rating_byhuman_twin_mean', 'creativity_rating_byhuman_partial_twin_mean', 'creativity_rating_byhuman_full_twin_mean']
→ std‐summary written with columns: ['DAT_perf_human_std', 'score_forwardflow_human_std', 'SSPT_human_std', 'SSPT_cosine_human_std', 'SSPT_standardized_human_std', 'SSPT_standardized_cosine_human_std', 'creativity_rating_byhuman_human_std', 'creativity_rating_byhuman_partial_human_std', 'creativity_rating_byhuman_full_human_std', 'DAT_perf_twin_std', '

In [70]:
import numpy as np
import pandas as pd
from scipy.stats import f, norm, pearsonr, ttest_rel

# --- 1) extract parallel arrays ---
dat_h = df_human["DAT_perf"].to_numpy()
dat_t = df_twin["DAT_perf"].to_numpy()
sspt_h_standardized = df_human["SSPT_standardized"].to_numpy()
sspt_t_standardized = df_twin["SSPT_standardized"].to_numpy()
sspt_h = df_human["SSPT"].to_numpy()
sspt_t = df_twin["SSPT"].to_numpy()
crea_hbyh = df_human["creativity_rating_byhuman"].to_numpy()
crea_tbyh = df_twin["creativity_rating_byhuman"].to_numpy()
crea_hbyt = df_human["creativity_rating_bytwins"].to_numpy()
crea_tbyt = df_twin["creativity_rating_bytwins"].to_numpy()


# define common in‐sample mask (non‐missing on all three)
mask_dat = ~np.isnan(dat_h) & ~np.isnan(dat_t)
mask_sspt = ~np.isnan(sspt_h) & ~np.isnan(sspt_t)
mask_crea = ~np.isnan(crea_hbyh) & ~np.isnan(crea_tbyh)
indsin = mask_dat & mask_sspt & mask_crea
n = indsin.sum()

# --- 2) MAPE for each measure ---
mape_dat = np.mean(np.abs(dat_h[indsin] - dat_t[indsin]) / dat_h[indsin])
mape_sspt = np.mean(np.abs(sspt_h[indsin] - sspt_t[indsin]) / sspt_h[indsin])
mape_crea = np.mean(np.abs(crea_hbyh[indsin] - crea_tbyh[indsin]) / crea_hbyh[indsin])
mape = np.array([mape_dat, mape_sspt, mape_crea])
print("MAPE (DAT, SSPT, creativity):", mape)

# --- 3) paired t-tests on means ---
n_pairs = indsin.sum()
df_pairs = n_pairs - 1

print("\nDAT paired t-test for means:")
t_dat, p_dat = ttest_rel(dat_h[indsin], dat_t[indsin])
print(f" t = {t_dat:.3f}, df = {df_pairs}, p = {p_dat:.3f}")

print("SSPT paired t-test for means:")
t_sspt, p_sspt = ttest_rel(sspt_h[indsin], sspt_t[indsin])
print(f" t = {t_sspt:.3f}, df = {df_pairs}, p = {p_sspt:.3f}")

print("creativity paired t-test for means:")
t_crea, p_crea = ttest_rel(crea_hbyh[indsin], crea_tbyh[indsin])
print(f" t = {t_crea:.3f}, df = {df_pairs}, p = {p_crea:.3f}")


# --- 4) two-sample F-test for variance ---
x_dat, y_dat = dat_h[indsin], dat_t[indsin]
df1_dat, df2_dat = len(x_dat) - 1, len(y_dat) - 1
F_dat, pF_dat = f_test(x_dat, y_dat)
print(
    f"\nDAT F-test for variance: F = {F_dat:.3f}, df1 = {df1_dat}, df2 = {df2_dat}, p = {pF_dat:.3f}"
)

x_sspt, y_sspt = sspt_h[indsin], sspt_t[indsin]
df1_sspt, df2_sspt = len(x_sspt) - 1, len(y_sspt) - 1
F_sspt, pF_sspt = f_test(x_sspt, y_sspt)
print(
    f"SSPT F-test for variance: F = {F_sspt:.3f}, df1 = {df1_sspt}, df2 = {df2_sspt}, p = {pF_sspt:.3f}"
)

x_crea, y_crea = crea_hbyh[indsin], crea_tbyh[indsin]
df1_crea, df2_crea = len(x_crea) - 1, len(y_crea) - 1
F_crea, pF_crea = f_test(x_crea, y_crea)
print(
    f"creativity F-test for variance: F = {F_crea:.3f}, df1 = {df1_crea}, df2 = {df2_crea}, p = {pF_crea:.3f}"
)


# --- 5) correlations ---
n_corr = indsin.sum()
df_corr = n_corr - 2
r_dat, pR_dat = pearsonr(dat_h[indsin], dat_t[indsin])
print(f"\nDAT correlation: r = {r_dat:.3f}, df = {df_corr}, p = {pR_dat:.3f}")
r_sspt, pR_sspt = pearsonr(sspt_h_standardized[indsin], sspt_t_standardized[indsin])
print(f"SSPT_standardized correlation: r = {r_sspt:.3f}, df = {df_corr}, p = {pR_sspt:.3f}")
r_crea, pR_crea = pearsonr(crea_hbyh[indsin], crea_tbyh[indsin])
print(f"creativity correlation: r = {r_crea:.3f}, df = {df_corr}, p = {pR_crea:.3f}")

# --- 6) z-tests comparing human vs. twin correlations ---


def z_test_diff_corr(r1, r2, n1, n2):
    z1 = np.arctanh(r1)
    z2 = np.arctanh(r2)
    se = np.sqrt(1 / (n1 - 3) + 1 / (n2 - 3))
    z = (z1 - z2) / se
    p = 2 * (1 - norm.cdf(abs(z)))
    return z, p


df_z1 = n - 3
df_z2 = n - 3

# DAT vs. SSPT
z_dat_sspt, pz_dat_sspt = z_test_diff_corr(
    pearsonr(dat_h[indsin], sspt_h_standardized[indsin])[0],
    pearsonr(dat_t[indsin], sspt_t_standardized[indsin])[0],
    n,
    n,
)
print(
    f"\ncorr(DAT, SSPT) human vs. twin: "
    f"z = {z_dat_sspt:.3f}, df1 = {df_z1}, df2 = {df_z2}, p = {pz_dat_sspt:.3f}"
)

# DAT vs. creativity (rated by human)
z_dat_crea, pz_dat_crea = z_test_diff_corr(
    pearsonr(dat_h[indsin], crea_hbyh[indsin])[0],
    pearsonr(dat_t[indsin], crea_tbyh[indsin])[0],
    n,
    n,
)
print(
    f"corr(DAT, creativity rated by human) human vs. twin: "
    f"z = {z_dat_crea:.3f}, df1 = {df_z1}, df2 = {df_z2}, p = {pz_dat_crea:.3f}"
)

# SSPT vs. creativity (rated by human)
z_sspt_crea, pz_sspt_crea = z_test_diff_corr(
    pearsonr(sspt_h_standardized[indsin], crea_hbyh[indsin])[0],
    pearsonr(sspt_t_standardized[indsin], crea_tbyh[indsin])[0],
    n,
    n,
)
print(
    f"corr(SSPT, creativity rated by human) human vs. twin: "
    f"z = {z_sspt_crea:.3f}, df1 = {df_z1}, df2 = {df_z2}, p = {pz_sspt_crea:.3f}"
)

# DAT vs. creativity (rated by same source)
z_dat_crea2, pz_dat_crea2 = z_test_diff_corr(
    pearsonr(dat_h[indsin], crea_hbyh[indsin])[0],
    pearsonr(dat_t[indsin], crea_tbyt[indsin])[0],
    n,
    n,
)
print(
    f"corr(DAT, creativity rated by same source) human vs. twin: "
    f"z = {z_dat_crea2:.3f}, df1 = {df_z1}, df2 = {df_z2}, p = {pz_dat_crea2:.3f}"
)

# SSPT vs. creativity (rated by same source)
z_sspt_crea2, pz_sspt_crea2 = z_test_diff_corr(
    pearsonr(sspt_h_standardized[indsin], crea_hbyh[indsin])[0],
    pearsonr(sspt_t_standardized[indsin], crea_tbyt[indsin])[0],
    n,
    n,
)
print(
    f"corr(SSPT, creativity rated by same source) human vs. twin: "
    f"z = {z_sspt_crea2:.3f}, df1 = {df_z1}, df2 = {df_z2}, p = {pz_sspt_crea2:.3f}"
)

MAPE (DAT, SSPT, creativity): [0.06439529 0.01174344 0.17660349]

DAT paired t-test for means:
 t = -12.168, df = 198, p = 0.000
SSPT paired t-test for means:
 t = 1.044, df = 198, p = 0.298
creativity paired t-test for means:
 t = -2.718, df = 198, p = 0.007

DAT F-test for variance: F = 7.957, df1 = 198, df2 = 198, p = 0.000
SSPT F-test for variance: F = 2.053, df1 = 198, df2 = 198, p = 0.000
creativity F-test for variance: F = 1.816, df1 = 198, df2 = 198, p = 0.000

DAT correlation: r = 0.068, df = 197, p = 0.343
SSPT_standardized correlation: r = 0.105, df = 197, p = 0.139
creativity correlation: r = 0.088, df = 197, p = 0.217

corr(DAT, SSPT) human vs. twin: z = -0.771, df1 = 196, df2 = 196, p = 0.441
corr(DAT, creativity rated by human) human vs. twin: z = -0.279, df1 = 196, df2 = 196, p = 0.780
corr(SSPT, creativity rated by human) human vs. twin: z = -1.661, df1 = 196, df2 = 196, p = 0.097
corr(DAT, creativity rated by same source) human vs. twin: z = 1.837, df1 = 196, df2 = 19